# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff
using PseudoPotentialData

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice = T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
    atoms     = [ElementPsp(:He, pseudopotentials)]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions;
                      functionals=[:lda_x, :lda_c_vwn],
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770852984465                   -0.53    9.0    173ms
  2   -2.772143045155       -2.89       -1.31    1.0    139ms
  3   -2.772170320480       -4.56       -2.57    1.0    163ms
  4   -2.772170689447       -6.43       -3.61    1.0    109ms
  5   -2.772170722402       -7.48       -4.10    2.0    141ms
  6   -2.772170722996       -9.23       -5.07    1.0    125ms
  7   -2.772170723014      -10.74       -5.57    1.0    119ms
  8   -2.772170723015      -12.08       -6.03    2.0    132ms
  9   -2.772170723015      -13.30       -6.86    1.0    124ms
 10   -2.772170723015   +  -14.10       -7.96    2.0    137ms
 11   -2.772170723015      -13.74       -8.85    1.0    124ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770769292987                   -0.53    9.0    172ms
  2   -2.7

1.7735580429064521

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem. This leads to a density-functional perturbation
theory problem, which is automatically set up and solved in the background.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770415118734                   -0.53    9.0    188ms
  2   -2.772050943916       -2.79       -1.29    1.0    121ms
  3   -2.772082734276       -4.50       -2.74    1.0    114ms
  4   -2.772083416407       -6.17       -4.16    2.0    130ms
  5   -2.772083417797       -8.86       -4.97    2.0    135ms
  6   -2.772083417810      -10.89       -5.88    1.0    120ms
  7   -2.772083417811      -12.02       -6.37    2.0    573ms
  8   -2.772083417811      -14.24       -6.57    1.0    114ms
  9   -2.772083417811      -13.82       -7.39    1.0    117ms
 10   -2.772083417811      -13.92       -8.14    2.0    149ms
Solving response problem
[ Info: GMRES linsolve starts with norm of residual = 4.19e+00
[ Info: GMRES linsolve in iteration 1; step 1: normres = 2.49e-01
[ Info: GMRES linsolve in iteration 1; step 2: normres = 3.76e-03
[ Info: GMRES linsolve in iteration 1